In [1]:
import sys
sys.path.append("/u/lukas.rilling/dev/")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import h5py
%matplotlib widget

from returnn_training_progress import get_epoch_data
from returnn_training_plot_nb import plot_df
%load_ext autoreload
%autoreload 2

In [2]:
globs = [
    "/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/flow_given_alignment/ga_glowTTS_ASR_ffn_x_vector*/200ep/*/ce*/phoneme_pred/train-clean/output/output.hdf",
    "/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/flow_given_alignment/ga_glowTTS_ASR_cnn_x_vector*/*/ce*/phoneme_pred/train-clean/output/output.hdf",
]
lr_files = []
for g in globs:
    lr_files += glob.glob(g)

print(lr_files)
exlude = "speaker_drop"
lr_files = [l for l in lr_files if not exlude in l]
# print(lr_files)
common_prefix = os.path.commonpath(lr_files)
common_sufix = os.path.commonpath([l[::-1] for l in lr_files])[::-1]

names = []
for f in lr_files:
    names.append(f.removeprefix(common_prefix).removesuffix(common_sufix))

files = dict(zip(lr_files, names))
files, len(lr_files)

['/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/flow_given_alignment/ga_glowTTS_ASR_ffn_x_vector/200ep/basic_init/ce_ls_0.1/phoneme_pred/train-clean/output/output.hdf', '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/flow_given_alignment/ga_glowTTS_ASR_ffn_x_vector/200ep/tts_pretrained/ce_ls_0.1/phoneme_pred/train-clean/output/output.hdf', '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/flow_given_alignment/ga_glowTTS_ASR_ffn_x_vector_v2/200ep/tts_pretrained/ce_ls_1.0/phoneme_pred/train-clean/output/output.hdf', '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/flow_given_alignment/ga_glowTTS_ASR_ffn_x_vector_v2/200ep/tts_pretrained/ce_ls_0.1/phoneme_pred/train

({'/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/flow_given_alignment/ga_glowTTS_ASR_ffn_x_vector/200ep/basic_init/ce_ls_0.1/phoneme_pred/train-clean/output/output.hdf': '/ga_glowTTS_ASR_ffn_x_vector/200ep/basic_init/ce_ls_0.1/',
  '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/flow_given_alignment/ga_glowTTS_ASR_ffn_x_vector/200ep/tts_pretrained/ce_ls_0.1/phoneme_pred/train-clean/output/output.hdf': '/ga_glowTTS_ASR_ffn_x_vector/200ep/tts_pretrained/ce_ls_0.1/',
  '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/flow_given_alignment/ga_glowTTS_ASR_ffn_x_vector_v2/200ep/tts_pretrained/ce_ls_1.0/phoneme_pred/train-clean/output/output.hdf': '/ga_glowTTS_ASR_ffn_x_vector_v2/200ep/tts_pretrained/ce_ls_1.0/',
  '/u/lukas.rilling/experiments/glow

In [3]:
# replace_dict = {
#     "librispeech_glow_asr/pytorch/encoding_test/": "",
#     "tts_architecture/glow_tts/raw_audio/decoder_test/": "",
#     "/": " | ",
#     "_encoder_sample_test": " Sampled ",
#     "_decoder_test": " Decoder output ",
#     # "_encoding_test_blstm": "1x512 BLSTM  Mean + log(std)",
#     "_encoding_test": "",
#     "_ce": "Mean + log(std)",
#     "_mean_only": "Mean",
#     "_encoding_test_mean_only": "Mean",
#     "glowTTS": "",
#     "_maxlike_alignment": " MAS ",
#     "_simple_linear": " 1x80 linear ",
#     "_multi_layer_ffn": " 3x512 linear ",
#     "_blstm": " 1x512 BLSTM ",
#     "tts_architecture": "",
#     "enc192": "192 channels",
#     "enc768": "768 channels",
# }

In [4]:
# sorting_definition_glow_model = ["192 channels", "768 channels"]
# sorting_definition_model = [" 1x80 linear ", " 3x512 linear ", " 1x512 BLSTM "]
# sorting_definition_mean = ["mean_only", "with_sigma"]
# sorting_definition_input = ["Mean |", "Mean + log(std) |", " Sampled  |", " Sampled  MAS  |", " Decoder output  |"]
# def sorting_function(a):
#     if "mean_only" in a[1] or "with_sigma" in a[1]:
#         split_a = a[1].split(" | ")

#         sorting_index_a = sorting_definition_model.index(split_a[0]) * 1e3

#         if "channels" in a[1]:
#             sorting_index_a += sorting_definition_glow_model.index(split_a[1]) * 1e2 + sorting_definition_mean.index(split_a[2]) * 1e1 + sorting_definition_input.index(split_a[3])   
#         else:
#             sorting_index_a += sorting_definition_mean.index(split_a[1]) * 1e1 + sorting_definition_input.index(split_a[2])

#     else:
#         split_a = a[1].split(" | ")

#         sorting_index_a = sorting_definition_model.index(split_a[0]) * 1e2
#         if "channels" in a[1]:
#             sorting_index_a += sorting_definition_glow_model.index(split_a[1]) * 1e1 + sorting_definition_input.index(split_a[2])
#         else:
#             sorting_index_a += sorting_definition_input.index(split_a[1])

#     return sorting_index_a

# for k,v in new_files.items():
#     if "with_sigma" in v:
#         sorting_function([k,v])
#         break

In [5]:
# sorted_files = dict(sorted(new_files.items(), key=sorting_function))
# sorted_files.values()

In [6]:
# print("| Model Type | Accuary | Cross Entropy |\n|-----|-----|-----|")
df = {}
for file, name in files.items():
    data = h5py.File(file)
    epoch_data = get_epoch_data(
        file.replace("phoneme_pred/train-clean/output/output.hdf", "training/work/learning_rates"), 100
    )
    if "dev_loss_ce" in epoch_data["error"]: 
        dev_loss_ce = epoch_data["error"]["dev_loss_ce"]
    else:
        dev_loss_ce = np.inf
    mean = np.array(data["inputs"]).mean()
    with open(file.replace("alias", "output").replace("phoneme_pred/train-clean/output/output.hdf", "tts_eval_gl/nisqa_mos/average")) as f:
        autoMOS = float(f.readline())
    df[name] = {"Model Name": name, "Accuracy": 100*mean, "CE": dev_loss_ce, "AutoMOS": autoMOS}
    # print(f"| {name} | {1e2*mean:.2f} | {dev_loss_ce:.3f} |")

df = pd.DataFrame.from_dict(df, orient="index")
print(df.to_markdown(index=False, floatfmt=".2f"))

| Model Name                                                      |   Accuracy |   CE |   AutoMOS |
|:----------------------------------------------------------------|-----------:|-----:|----------:|
| /ga_glowTTS_ASR_ffn_x_vector/200ep/basic_init/ce_ls_0.1/        |      21.39 | 2.94 |      1.92 |
| /ga_glowTTS_ASR_ffn_x_vector/200ep/tts_pretrained/ce_ls_0.1/    |      21.33 | 2.94 |      2.73 |
| /ga_glowTTS_ASR_ffn_x_vector_v2/200ep/tts_pretrained/ce_ls_1.0/ |      20.42 | 0.02 |      2.05 |
| /ga_glowTTS_ASR_ffn_x_vector_v2/200ep/tts_pretrained/ce_ls_0.1/ |      20.22 | 0.14 |      2.06 |
| /ga_glowTTS_ASR_ffn_x_vector_v2/200ep/basic_init/ce_ls_1.0/     |      20.48 | 0.01 |      2.06 |
| /ga_glowTTS_ASR_ffn_x_vector_v2/200ep/basic_init/ce_ls_0.1/     |      20.71 | 0.13 |      2.06 |
| /ga_glowTTS_ASR_cnn_x_vector/basic_init/ce_ls_0.1/              |      25.67 | 2.90 |      2.40 |
| /ga_glowTTS_ASR_cnn_x_vector/tts_pretrained/ce_ls_0.1/          |      27.63 | 2.89 |      2.34 |
